In [31]:
import torch 
import torch.nn as nn 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import torch.nn.functional as F
from tqdm import tqdm
import os 
from torchinfo import summary
from torchvision.io import read_image
from torch.utils.data import random_split
import time 
import random # Random Search 
from sklearn.metrics import accuracy_score
from torch.optim import Adam

In [7]:
path_to_data = "/run/media/magnusjsc/T7/Classification-and-3D-reconstruction-of-archaeological-artifacts_DATA/DIME images"

def load_data_from_directory(directory_path, label, limit = 10000):
    data = []
    labels = []
    count = 0 

    for filename in os.listdir(directory_path):
        if filename.endswith(".jpg"):
            image_path = os.path.join(directory_path, filename)
            image = read_image(image_path)

            # Check if image is RGB
            if image.shape[0] == 3: 
                data.append(image)
                labels.append(label)
                count += 1 

            if count >= limit: 
                break; 

    return data, labels # Tuple 

# Load the data 
data, labels = load_data_from_directory(
    path_to_data,
    label = 0, # CHANGE
    limit = 25 
)

In [23]:
imgTrans = transforms.Compose(
    [
        # Add more if needed
        transforms.ToPILImage(),
        transforms.ToTensor()
        # TODO: ADD RESIZE!
    ]
)

training = 0.7
validation = 0.15
testing = 0.15 

# TODO: MISSING LABELS
train_set, val_set, test_set = random_split(data, [training, validation, testing]) # Partition the dataset

batch_size = 50

train_loader = DataLoader(train_set, batch_size = batch_size, shuffle = True)
val_loader = DataLoader(val_set, batch_size = batch_size, shuffle = False)
test_loader = DataLoader(test_set, batch_size = batch_size, shuffle = False) 

print(f'Length of training set: {len(train_set)}, validation set: {len(val_set)}, testing set: {len(test_set)}')

n_channels = 3
n_classes = 1 # TODO: Calculate number of distinct classes

Length of training set: 18, validation set: 4, testing set: 3


In [35]:
# Classifier  
class CNN(nn.Module):
    def __init__(
        self, 
        num_layers: int 
    ):
        super(CNN, self).__init__()

        self.num_layers = num_layers
        self.layers = nn.ModuleList() 

        for i in range(num_layers):
            in_channels = n_channels if i == 0 else 64 # TODO: Fix 64 to be more generic  
            out_channels = 64 # Make dynamic 
            self.layers.append(
                nn.Conv2d(
                    in_channels = in_channels,
                    out_channels = out_channels,
                    kernel_size = 3,
                    padding = 1
                )
            )
            self.layers.append(nn.ReLU())
            self.layers.append(
                nn.MaxPool2d(
                    kernel_size = 2,
                    stride = 2
                )
            )

        input_size = 64 # TODO: Change 
            
        self.fc = nn.Linear(64 * (input_size // 2 ** num_layers) * (input_size // 2 ** num_layers), n_classes)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = x.view(x.size(0), -1) # Flatten
        x = self.fc(x)
        return x 

criterion = nn.BCEWithLogitsLoss() 
cnn_model = CNN(
    num_layers = 3
)

optimizer = Adam(cnn_model.parameters(), lr = .003)

summary(
    cnn_model,
    input_size = (batch_size, 3, 224, 224), 
    verbose = 2, 
    col_names = [
        "input_size", 
        "num_params", 
        "output_size", 
        "mult_adds", 
        "trainable"
    ],
    mode = 'train'
)

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: [Conv2d: 2, ReLU: 2, MaxPool2d: 2, Conv2d: 2, ReLU: 2, MaxPool2d: 2, Conv2d: 2, ReLU: 2, MaxPool2d: 2]

In [ ]:
# Training, validation and testing loop 

# TODO !!! 

In [22]:
# Random Search for hyperparameter search
hyperparams = {
    'learning_rate': [0.001, 0.01, 0.1],
    'batch_size': [16, 32, 64],
    'num_epochs': [30, 90, 120],
    'num_layers': [2, 3, 4, 5],
    'layer_out': [32, 64, 128],
    'kernel_size': [0, 2, 4],
    # TODO: Add more! 
}

trails = 10 

best_accuracy = 0
best_hyperparams = None

# TODO: Finish random search function